In [ ]:
!pip3 install scikit-plot

In [ ]:
import keras 
import seaborn as sns
import scikitplot
from keras.models import Sequential
from matplotlib import pyplot
from keras.layers import Dense,Flatten,Dropout
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator ,load_img
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau

%matplotlib inline
import os
import requests
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split

In [ ]:
#Importing Data from CSV file
fer_dataset = pd.read_csv("./fer2013.csv")
fer_dataset.head()

In [ ]:
emotion_labels=fer_dataset.iloc[:,[0]].values

In [ ]:
pixels=fer_dataset['pixels']

In [ ]:
#Facial Expressions
Expressions = {0:"Angry",1:"Disgust",2:"Fear",3:"Happy",4:"Sad",5:"Surprise",6:"Neutral"}
from keras.utils import to_categorical 
emotion_labels = to_categorical(emotion_labels,len(Expressions))
print(emotion_labels)

In [ ]:
sns.countplot(fer_dataset.emotion)
pyplot.show()

In [ ]:
#converting pixels to Gray Scale images of 48X48 

images = np.array([np.fromstring(pixel, dtype=int, sep=" ")for pixel in pixels])
images = images/255.0
images = images.reshape(images.shape[0],48,48,1).astype('float32')


In [ ]:
plt.imshow(images[0][:,:,0])
Expressions[emotion_labels[0][0]]

In [ ]:
#splitting data into training and test data
train_images,test_images,train_labels,test_labels = train_test_split(images,emotion_labels,test_size=0.2,random_state=0)

In [ ]:
train_labels

In [ ]:
def create_convolutional_model(emotions):
    seq_model = Sequential()
    seq_model.add(Conv2D(32,kernel_size=(2,2),strides=(1,1),activation='relu',input_shape=(48,48,1)))
    seq_model.add(BatchNormalization())
    seq_model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    seq_model.add(Dropout(0.25))
    
    seq_model.add(Conv2D(filters=64,kernel_size=(2,2),strides=(1,1),activation='relu'))
    seq_model.add(BatchNormalization())
    seq_model.add(MaxPooling2D(pool_size=(2,2),strides=(1,1)))
    seq_model.add(Dropout(0.25))#to prevent neural network from overfitting
    
    seq_model.add(Conv2D(filters=128,kernel_size=(2,2),strides=(1,1),activation='relu'))
    seq_model.add(BatchNormalization())
    seq_model.add(MaxPooling2D(pool_size=(2,2),strides=(1,1)))
    seq_model.add(Dropout(0.25))
    
    seq_model.add(Conv2D(filters=256,kernel_size=(2,2),strides=(1,1),activation='relu'))
    seq_model.add(BatchNormalization())
    seq_model.add(MaxPooling2D(pool_size=(2,2),strides=(1,1)))
    seq_model.add(Dropout(0.25))
    
    seq_model.add(Flatten())
    
    seq_model.add(Dense(256,activation='relu'))
    seq_model.add(BatchNormalization())
    seq_model.add(Dropout(0.25))
    
    seq_model.add(Dense(512,activation='relu'))
    seq_model.add(BatchNormalization())
    seq_model.add(Dropout(0.25))
    
    seq_model.add(Dense(emotions,activation='softmax')) 

    seq_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return seq_model 

In [ ]:
learn_rate = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    learn_rate,
]

In [ ]:
# As the data in hand is less as compared to the task so ImageDataGenerator is good to go.
data_augment = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
)
data_augment.fit(train_images)

In [ ]:
emotions=7
cnn_model = create_convolutional_model(emotions)
cnn_model.summary()

In [ ]:
#train the CNN 
history = cnn_model.fit_generator(
    data_augment.flow(train_images, train_labels, batch_size=105),
    validation_data=(test_images, test_labels),
    epochs=110,
    callbacks=callbacks,verbose=2
)



In [ ]:
emotion_pred = cnn_model.predict(test_images)
emotion_pred=np.argmax(emotion_pred,axis = 1)

In [ ]:
import seaborn as sns
from matplotlib import pyplot

sns.set()
fig = pyplot.figure(0, (12, 4))

ax = pyplot.subplot(1, 2, 1)
sns.lineplot(history.epoch, history.history['accuracy'], label='train')
sns.lineplot(history.epoch, history.history['val_accuracy'], label='test')
pyplot.title('Accuracy Graph')
pyplot.tight_layout()

ax = pyplot.subplot(1, 2, 2)
sns.lineplot(history.epoch, history.history['loss'], label='train')
sns.lineplot(history.epoch, history.history['val_loss'], label='test')
pyplot.title('Loss Graph')
pyplot.tight_layout()

pyplot.savefig('epoch_history_dcnn.png')
pyplot.show()

In [ ]:
#making confusion matrix
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, emotions, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(emotions))
    plt.xticks(tick_marks, emotions, rotation=45)
    plt.yticks(tick_marks, emotions)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

test_labels=np.argmax(test_labels,axis=1)
cnf_matrix = confusion_matrix(test_labels,emotion_pred)
class_names=Expressions
# Plotting normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, emotions=class_names, normalize=True,
                      title='Normalized confusion matrix')
plt.show()

In [ ]:
#Save the weights
weight_file='model_weights.hdf5'
cnn_model.save_weights(weight_file,overwrite=True)

In [ ]:
#Save the weights
fileweight_filename='model_weights.hdf5'
cnn_model.load_weights(weight_file)

In [ ]:
# making folders

import os 
outer_file = ['result']
inner_file = ['Angry', 'Disgusted', 'Fearful', 'Happy', 'Sad', 'Surprise', 'Neutral']
os.makedirs('data', exist_ok=True)
for outer in outer_file:
    os.makedirs(os.path.join('data',outer), exist_ok=True)
    for inner in inner_file:
        os.makedirs(os.path.join('data',outer,inner), exist_ok=True)



In [ ]:
import cv2
def make_prediction(images):
    images=cv2.resize(images,(48,48))
    images=images/255.0
    images=np.array(images).reshape(-1,48,48,1)
    predict=np.argmax(cnn_model.predict(images),axis = 1)
    return predict[0]  

In [ ]:
from google.colab.patches import cv2_imshow

def face_in_img(imagePath):
    face_cascadeFile = cv2.CascadeClassifier("./haarcascade_frontalface_default.xml")
    img = cv2.imread(imagePath) 
    grayImage = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_cascadeFile.detectMultiScale(grayImage,1.3,5)

    for (x,y,w,h) in faces:
            crop_face = grayImage[y:y+h, x:x+w]
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            res=make_prediction(crop_face)
            font = cv2.FONT_HERSHEY_SIMPLEX
            label_img=cv2.putText(img,str(Expressions[res]),(x,y-5),font,0.7,(205,200,50),2,cv2.LINE_AA)

            if res == 0:
              cv2.imwrite('./data/result/Angry/im'+Expressions[res]+'.png',label_img)
            if res == 1:
              cv2.imwrite('./data/result/Disgust/im'+Expressions[res]+'.png',label_img)
            if res == 2:
              cv2.imwrite('./data/result/Fear/im'+Expressions[res]+'.png',label_img)
            if res == 3:
              cv2.imwrite('./data/result/Happy/im'+Expressions[res]+'.png',label_img)
            if res == 4:
              cv2.imwrite('./data/result/Sad/im'+Expressions[res]+'.png',label_img)
            if res == 5:
              cv2.imwrite('./data/result/Surprise/im'+Expressions[res]+'.png',label_img)
            if res == 6:
              cv2.imwrite('./data/result/Neutral/im'+Expressions[res]+'.png',label_img)
                 
    cv2_imshow(img)


In [ ]:
from PIL import Image
import os, sys
from google.colab.patches import cv2_imshow

path = './test_images'
dirs = os.listdir(path)

for item in dirs:
  if path+item:
    im= cv2.imread(path+'/'+item) 
    face_in_img(path+'/'+item)

In [ ]:
%matplotlib inline
import os
import requests
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [ ]:
anotationFile='./instances_val2017.json'

In [ ]:
cocodata=COCO(anotationFile)

In [ ]:
# display COCO categories and supercategories
cats = cocodata.loadCats(cocodata.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

In [ ]:
# get all images containing given categories, select one at random
categ_Ids = cocodata.getCatIds(catNms=['person']);
imageIds = cocodata.getImgIds(catIds=categ_Ids );
images = cocodata.loadImgs(imageIds)
print("Number of images containing all the  classes:", len(imageIds))

img = cocodata.loadImgs(imageIds[np.random.randint(0,len(imageIds))])[0]

In [ ]:
# use url to load image
I = io.imread(img['coco_url'])
plt.axis('off')
plt.imshow(I)
plt.show()

In [ ]:
#Loading images in a folder
os.makedirs('coco_images', exist_ok=True)


In [ ]:
for im in images:
    #print("im: ", im)
    coco_img = requests.get(im['coco_url']).content
    with open('coco_images/' + im['file_name'], 'wb') as handler:
        handler.write(coco_img)

In [ ]:
from PIL import Image
import os, sys
from google.colab.patches import cv2_imshow

coco_imgpath = './coco_images'
dirs = os.listdir(coco_imgpath)

for item in dirs:
  if coco_imgpath+item:
    im= cv2.imread(coco_imgpath+'/'+item) 
    face_in_img(coco_imgpath+'/'+item)



In [ ]:
## References


# https://medium.com/themlblog/how-to-do-facial-emotion-recognition-using-a-cnn-b7bbae79cd8f
# https://github.com/serengil/tensorflow-101/blob/master/python/facial-expression-recognition.py
# https://www.kaggle.com/jonathanoheix/face-expression-recognition-dataset
# https://analyticsindiamag.com/my-first-cnn-project-emotion-detection-using-convolutional-neural-network-with-tpu/